## Mã hóa đếm/tần số

Trong mã hóa đếm, chúng ta thay thế các hạng mục bằng số lượng quan sát hiển thị hạng mục đó trong tập dữ liệu. Tương tự, chúng ta có thể thay thế hạng mục bằng tần số - hoặc tỷ lệ phần trăm - của các quan sát trong tập dữ liệu. Nghĩa là, nếu có 10 trong số 100 quan sát hiển thị blue thì chúng ta sẽ thay thế blue bằng 10 nếu thực hiện mã hóa đếm hoặc 0.1 nếu thay thế bằng tần số. Các kỹ thuật này nắm bắt biểu diễn của mỗi nhãn trong tập dữ liệu, nhưng mã hóa có thể không nhất thiết dự đoán kết quả. Tuy nhiên, đây là những phương pháp mã hóa rất phổ biến trong các cuộc thi Kaggle.

Giả định của kỹ thuật này là số quan sát được hiển thị theo mỗi biến phần nào cung cấp thông tin về khả năng dự đoán của hạng mục.


### Ưu điểm

- Đơn giản.
- Không mở rộng không gian đặc trưng.

### Hạn chế

- Nếu 2 hạng mục khác nhau có cùng số lần xuất hiện trong tập dữ liệu, tức là chúng có số lượng quan sát giống nhau thì sẽ được thay thế bằng cùng một số: có thể mất thông tin có giá trị.

Ví dụ: nếu có 10 quan sát cho hạng mục blue và 10 quan sát cho hạng mục red, cả hai sẽ được thay thế bằng 10 nên sau khi mã hóa sẽ xuất hiện cùng một thứ.


Theo dõi [thread trong Kaggle](https://www.kaggle.com/general/16927) để biết thêm thông tin.



## Trong bản mô phỏng này:

Chúng ta sẽ thực hiện mã hóa đếm/tần số với:
- pandas
- Feature-Engine

Chúng ta sẽ sử dụng tập dữ liệu giá nhà để minh họa các ưu điểm và hạn chế của từng triển khai.

In [1]:
import numpy as np
import pandas as pd

# chia tập dữ liệu
from sklearn.model_selection import train_test_split

# mã hóa với feature-engine
from feature_engine.encoding import CountFrequencyEncoder

In [3]:
# load tập dữ liệu

data = pd.read_csv(
    './datatset/house-price/houseprice.csv',
    usecols=['Neighborhood', 'Exterior1st', 'Exterior2nd', 'SalePrice'])

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [4]:
# xem từng biến có bao nhiêu nhãn

for col in data.columns:
    print(col, ': ', len(data[col].unique()), ' labels')

Neighborhood :  25  labels
Exterior1st :  15  labels
Exterior2nd :  16  labels
SalePrice :  663  labels


### Quan trọng

Khi thực hiện phép biến đổi đếm của các biến hạng mục, cần tính số lượng (hoặc tần số = số lượng/tổng số quan sát) **trên tập huấn luyện**, sau đó sử dụng các số đó để thay thế các nhãn trong tập kiểm tra.

In [5]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Mã hóa đếm/tần số với pandas

In [8]:
## Yêu cầu 1: lấy số đếm cho từng nhãn
# trong biến Neigbourhood

## VIẾT CODE Ở ĐÂY:
count_map = X_train['Neighborhood'].value_counts()

count_map

NAmes      151
CollgCr    105
OldTown     73
Edwards     71
Sawyer      61
Somerst     56
Gilbert     55
NridgHt     51
NWAmes      51
SawyerW     45
BrkSide     41
Mitchel     36
Crawfor     35
Timber      30
NoRidge     30
ClearCr     24
IDOTRR      24
SWISU       18
StoneBr     16
Blmngtn     12
MeadowV     12
BrDale      10
NPkVill      7
Veenker      6
Blueste      2
Name: Neighborhood, dtype: int64

<details><summary> Gợi ý </summary>

[value_counts()](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)

[to_dict()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_dict.html)

</details>

Dictionary chứa số quan sát trên mỗi hạng mục trong biến Neighbourhood.

In [9]:
## Yêu cầu 2: thay các nhãn bằng số đếm

## VIẾT CODE Ở ĐÂY:
X_train['Neighborhood'] = X_train['Neighborhood'].map(count_map)
X_test['Neighborhood'] = X_test['Neighborhood'].map(count_map)

In [10]:
# khám phá kết quả

X_train['Neighborhood'].head(10)

64      105
682      24
960      41
1384     71
1100     18
416      61
1034     35
853     151
472      71
1011     71
Name: Neighborhood, dtype: int64

In [20]:
## Yêu cầu 3: thay vì số đếm, chúng ta dùng tần số
# chúng ta chỉ cần chia số đếm cho tống số quan sát: 

## VIẾT CODE Ở ĐÂY:
frequency_map = (X_train['Exterior1st'].value_counts()/X_train['Exterior1st'].value_counts().sum())
frequency_map

VinylSd    0.356164
HdBoard    0.149706
Wd Sdng    0.144814
MetalSd    0.135029
Plywood    0.084149
CemntBd    0.038160
BrkFace    0.034247
WdShing    0.020548
Stucco     0.016634
AsbShng    0.014677
Stone      0.001957
ImStucc    0.000978
AsphShn    0.000978
CBlock     0.000978
BrkComm    0.000978
Name: Exterior1st, dtype: float64

In [21]:
## Yêu cầu 4: thay các nhãn bằng tần số

## VIẾT CODE Ở ĐÂY:
X_train['Exterior1st'] = X_train['Exterior1st'].map(frequency_map)
X_test['Exterior1st'] = X_test['Exterior1st'].map(frequency_map)

In [22]:
def amount_frequency_feature(X_train,variable):
    return (X_train['Exterior1st'].value_counts()/X_train['Exterior1st'].value_counts().sum())

In [23]:
def map_labels_frequency(X_train,X_test,variable,frequency_maps):
    X_train[variable] = X_train[variable].map(frequency_maps)
    X_test[variable] = X_test[variable].map(frequency_maps)

Sau đó, chúng ta có thể đặt các lệnh này thành 2 hàm như đã làm trong 3 notebook trước và lặp lại trên tất cả các biến hạng mục. Nếu không biết cách làm điều này, vui lòng xem lại notebook trước đó.

## Mã hóa đếm/tần số với Feature-Engine

In [24]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['Neighborhood', 'Exterior1st', 'Exterior2nd']], # các yếu tố dự báo
    data['SalePrice'],  # mục tiêu
    test_size=0.3,  # phần trăm các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((1022, 3), (438, 3))

In [25]:
count_enc = CountFrequencyEncoder(
    encoding_method='count', # để thực hiện tần số ==> encoding_method='frequency'
    variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

count_enc.fit(X_train)

CountFrequencyEncoder(variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

In [26]:
# trong dict encoder, chúng ta thấy các số lượng
# các quan sát trên mỗi hạng mục cho từng biến

count_enc.encoder_dict_

{'Neighborhood': {'NAmes': 151,
  'CollgCr': 105,
  'OldTown': 73,
  'Edwards': 71,
  'Sawyer': 61,
  'Somerst': 56,
  'Gilbert': 55,
  'NridgHt': 51,
  'NWAmes': 51,
  'SawyerW': 45,
  'BrkSide': 41,
  'Mitchel': 36,
  'Crawfor': 35,
  'Timber': 30,
  'NoRidge': 30,
  'ClearCr': 24,
  'IDOTRR': 24,
  'SWISU': 18,
  'StoneBr': 16,
  'Blmngtn': 12,
  'MeadowV': 12,
  'BrDale': 10,
  'NPkVill': 7,
  'Veenker': 6,
  'Blueste': 2},
 'Exterior1st': {'VinylSd': 364,
  'HdBoard': 153,
  'Wd Sdng': 148,
  'MetalSd': 138,
  'Plywood': 86,
  'CemntBd': 39,
  'BrkFace': 35,
  'WdShing': 21,
  'Stucco': 17,
  'AsbShng': 15,
  'Stone': 2,
  'ImStucc': 1,
  'AsphShn': 1,
  'CBlock': 1,
  'BrkComm': 1},
 'Exterior2nd': {'VinylSd': 353,
  'Wd Sdng': 142,
  'HdBoard': 141,
  'MetalSd': 136,
  'Plywood': 112,
  'CmentBd': 39,
  'Wd Shng': 29,
  'BrkFace': 18,
  'AsbShng': 17,
  'Stucco': 16,
  'ImStucc': 8,
  'Stone': 4,
  'Brk Cmn': 4,
  'AsphShn': 1,
  'CBlock': 1,
  'Other': 1}}

In [27]:
X_train = count_enc.transform(X_train)
X_test = count_enc.transform(X_test)

# khám phá kết quả
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,105,364,353
682,24,148,142
960,41,148,112
1384,71,21,29
1100,18,148,142


**Lưu ý**

Nếu các biến đối số được để thành None thì encoder sẽ tự động xác định tất cả các biến hạng mục. Tuyệt đúng không?

Encoder sẽ không mã hóa các biến dạng số. Vì vậy, nếu một số biến dạng số là biến hạng mục thì chúng ta sẽ cần ép kiểu lại chúng thành object trước khi sử dụng encoder.

Nếu không có biến trong tập kiểm tra mà encoder không có số để chỉ định (không thấy hạng mục trong tập huấn luyện) thì encoder sẽ trả về lỗi.